In [12]:
import os
import glob
from pathlib import Path
import shutil, yaml
import random
from sklearn.model_selection import train_test_split
import numpy as np
from collections import Counter, defaultdict

In [13]:
%cd dataset

[Errno 2] No such file or directory: 'dataset'
/mnt/sdb/home/si25_osts0525/UAV_Inspection_System/dataset/target


In [15]:
# !curl -L "https://app.roboflow.com/ds/eyXcy3B9ln?key=XBBTNUflWL" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip
!curl -L "https://app.roboflow.com/ds/QuET3b01U2?key=9lItKMhdHz" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   910  100   910    0     0   3554      0 --:--:-- --:--:-- --:--:--  3540
100  321M  100  321M    0     0  57.1M      0  0:00:05  0:00:05 --:--:-- 69.8M
Archive:  roboflow.zip
  inflating: README.dataset.txt      
  inflating: README.roboflow.txt     
  inflating: data.yaml               
   creating: test/
   creating: test/images/
 extracting: test/images/000000_jpg.rf.0a86ce167f02d8a7c2b1688672884fbf.jpg  
 extracting: test/images/000001_jpg.rf.c1635fede493d75eafc461d8ace07009.jpg  
 extracting: test/images/000002_jpg.rf.978c681a879723b89d57b529fafba97b.jpg  
 extracting: test/images/000003_jpg.rf.0b01b7860c94b43da74657738a6b8fba.jpg  
 extracting: test/images/000004_jpg.rf.8d5a64f3ac69ce34a79a099ae68d0e10.jpg  
 extracting: test/images/000005_jpg.rf.e0d388f59bc6e5906b9a8d1da403cc24.jpg  
 extracting: test/images/000006_jp

In [16]:
##Helper functions

#Use to set up directory structure
def ensure_layout(root, splits):
    for split in splits:
        (root / split / "images").mkdir(parents=True, exist_ok=True)
        (root / split / "labels").mkdir(parents=True, exist_ok=True)

def first_class_id(label_file: Path) -> int:
    """
    Reads the first line of a YOLO label file and returns the class id (int).
    Assumes there is at least one object in the file.
    """
    with open(label_file, "r") as f:
        line = f.readline().strip()
    return int(line.split()[0])

def collect_labeled_images(split_dir: Path):
    """
    Returns a list of tuples: (image_path, label_path, class_id)
    Only keeps images that have a matching label file.
    """
    images_dir = split_dir / "images"
    labels_dir = split_dir / "labels"
    rows = []
    for img_path in sorted(images_dir.glob("*.*")):
        lbl = labels_dir / f"{img_path.stem}.txt"
        if lbl.exists():
            try:
                cid = first_class_id(lbl)
                rows.append((img_path, lbl, cid))
            except Exception:
                pass  # skip malformed label files
    return rows

def stratified_3way_split_with_minimum(class_ids,
                                        split_ratios=(0.8, 0.1, 0.1),
                                        min_instances=5,
                                        seed=42):
    """
    Return three Numpy index arrays (train, val, test) with
    roughly the given ratios while trying to keep class
    proportions equal (stratified).  Any class that has fewer
    than 'min_instances' samples in a split is pushed back into the *train*
    split so that `train_test_split(..., stratify=...)` does not
    throw a ValueError.
    """
    assert abs(sum(split_ratios) - 1.0) < 1e-6, "Ratios must sum to 1"
    rng = np.random.RandomState(seed)

    class_ids = np.asarray(class_ids)
    all_idx   = np.arange(len(class_ids))

    # ---------- Stage-0 : put absolute singletons to train ----------
    counts = Counter(class_ids)
    singleton_mask = np.array([counts[c] < min_instances for c in class_ids])
    train_idx = all_idx[singleton_mask]
    remain_idx = all_idx[~singleton_mask]
    remain_cls = class_ids[remain_idx]

    # ---------- Stage-1 : train vs tmp (val + test) ----------
    remain_train_ratio = (
        split_ratios[0] * len(class_ids) - len(train_idx)
    ) / len(remain_idx)

    tr_idx_part, tmp_idx = train_test_split(
        remain_idx,
        test_size=1 - remain_train_ratio,
        stratify=remain_cls,
        random_state=seed,
    )
    train_idx = np.concatenate([train_idx, tr_idx_part])

    # ---------- Stage-2 : assure no singleton inside tmp ----------
    tmp_cls = class_ids[tmp_idx]
    tmp_counts = Counter(tmp_cls)
    # indices inside tmp that belong to a class occurring once
    lone_in_tmp = [i for i, c in zip(tmp_idx, tmp_cls) if tmp_counts[c] == 1]
    if lone_in_tmp:
        train_idx = np.concatenate([train_idx, lone_in_tmp])
        tmp_idx   = np.setdiff1d(tmp_idx, lone_in_tmp)
        tmp_cls   = class_ids[tmp_idx]

    # ---------- Stage-3 : split tmp into val / test ----------
    val_ratio_within_tmp = split_ratios[1] / (split_ratios[1] + split_ratios[2])
    try:
        val_rel, test_rel = train_test_split(
            np.arange(len(tmp_idx)),
            test_size=1 - val_ratio_within_tmp,
            stratify=tmp_cls,
            random_state=seed,
        )
    except ValueError:
        # fallback: same split ratios but *without* stratification
        val_rel, test_rel = train_test_split(
            np.arange(len(tmp_idx)),
            test_size=1 - val_ratio_within_tmp,
            random_state=seed,
            shuffle=True,
        )

    # Convert indices to integers to avoid TypeError
    val_idx = tmp_idx[val_rel].astype(int)
    test_idx = tmp_idx[test_rel].astype(int)

    # keep indices sorted (optional, nice for reproducibility)
    return np.sort(train_idx), np.sort(val_idx), np.sort(test_idx)


def move_rows(rows, dst_root: Path):
    """
    Move (image,label) pairs into dst_root/{images,labels}.
    Skips files if they already exist in the destination directory.
    """
    for img_path, lbl_path, _ in rows:
        img_dst = dst_root / "images" / img_path.name
        lbl_dst = dst_root / "labels" / lbl_path.name
        
        # Check if image exists in source
        if not img_path.exists():
            print(f"WARNING: Image {img_path.name} does not exist in source directory.")
            continue  # Skip moving this file if it doesn't exist
        
        # Check if label exists in source
        if not lbl_path.exists():
            print(f"WARNING: Label {lbl_path.name} does not exist in source directory.")
            continue  # Skip moving this file if it doesn't exist

        # Check if image already exists in destination directory
        if not img_dst.exists():
            try:
                shutil.move(str(img_path), img_dst)
            except Exception as e:
                print(f"Error moving image {img_path.name}: {e}")
        else:
            print(f"Image {img_path.name} already exists in destination. Skipping move.")

        # Check if label already exists in destination directory
        if not lbl_dst.exists():
            try:
                shutil.move(str(lbl_path), lbl_dst)
            except Exception as e:
                print(f"Error moving label {lbl_path.name}: {e}")
        else:
            print(f"Label {lbl_path.name} already exists in destination. Skipping move.")


# Get image and label pairs for counts 
def get_image_label_pairs(base_dir):
    """
    Gathers all images and corresponding YOLO .txt label files from base_dir.
    Returns a list of dicts: [
      {'image_path': ..., 'label_path': ..., 'filename': ...}, ...
    ].
    """
    image_dir = os.path.join(base_dir, "images")
    label_dir = os.path.join(base_dir, "labels")

    # Collect possible image extensions
    img_extensions = ["*.jpg", "*.jpeg", "*.png", "*.bmp", "*.tif"]
    image_paths = []
    for ext in img_extensions:
        image_paths.extend(glob.glob(os.path.join(image_dir, ext)))

    data_pairs = []
    for img_path in image_paths:
        stem = os.path.splitext(os.path.basename(img_path))[0]
        label_path = os.path.join(label_dir, f"{stem}.txt")
        data_pairs.append({
            "image_path": img_path,
            "label_path": label_path if os.path.exists(label_path) else None,
            "filename": stem
        })
    return data_pairs


In [19]:
datasets_root = Path.cwd()  # directory containing original train/ val/ test/ (SOURCE/TARGET/TEST)
random.seed(42)

# # -------------------Rename 'valid' to 'val'-------------------
val_dir = datasets_root / "valid"
if val_dir.exists():
    print("Renaming 'valid' to 'val'")
    val_dir.rename(datasets_root / "val")

# # Set destination roots
source_root = datasets_root / "source"
target_root = datasets_root / "target"

# # Create directory structure
ensure_layout(source_root, splits=("train", "val", "test"))
# ensure_layout(target_root, splits=("train", "val", "test"))

# -------------------SOURCE: stratified 80/10/10 from original train-------------------
print("\nBuilding SOURCE domain (from original train/)...")
src_rows = collect_labeled_images(datasets_root / "train")

if len(src_rows) == 0:
    raise RuntimeError("No labeled images found in original train/")

cls_src = [r[2] for r in src_rows]  # Collect class ids

# Retrieve the index for each split after doing stratified split with minimum instances per class
# Ensure that every class has at least 5 instances in the train, val, and test sets
idx_train, idx_val, idx_test = stratified_3way_split_with_minimum(cls_src, split_ratios=(0.7, 0.15, 0.15), min_instances=5, seed=42)

# Move rows/data to the correct directory
move_rows([src_rows[i] for i in idx_train], source_root / "train")
move_rows([src_rows[i] for i in idx_val], source_root / "val")
move_rows([src_rows[i] for i in idx_test], source_root / "test")

print(f" SOURCE split: train={len(idx_train)}, val={len(idx_val)}, test={len(idx_test)}")

# -------------------TARGET: original val → stratified 80/20; original test → test -------------------
print("\nBuilding TARGET domain (from original val/ & test/)...")
tgt_rows_val = collect_labeled_images(datasets_root / "val")
tgt_rows_test = collect_labeled_images(datasets_root / "test")
tgt_rows = tgt_rows_val + tgt_rows_test  # merged pool

if len(tgt_rows) == 0:
    raise RuntimeError("No labeled images found in original val/ or test/")

cls_tgt = [r[2] for r in tgt_rows]  # Collect class ids

# Retrieve the index for each split after doing stratified split with minimum instances per class
# Ensure that every class has at least 5 instances in the train, val, and test sets
idx_train, idx_val, idx_test = stratified_3way_split_with_minimum(cls_tgt, split_ratios=(0.7, 0.15, 0.15), min_instances=5, seed=42)

# Move rows/data to the correct directory
move_rows([tgt_rows[i] for i in idx_train], target_root / "train")
move_rows([tgt_rows[i] for i in idx_val], target_root / "val")
move_rows([tgt_rows[i] for i in idx_test], target_root / "test")

print(f"TARGET: train={len(idx_train)}  val={len(idx_val)}  test={len(idx_test)}")

# Cleanup original roboflow files
print("\nCleaning up original RoboFlow folders...")
# for old in ("train", "val", "test"):
#     shutil.rmtree(datasets_root / old, ignore_errors=True)
for file in ("data.yaml", "README.dataset.txt", "README.roboflow.txt"):
    fp = datasets_root / file
    if fp.exists():
        os.remove(fp)



Building SOURCE domain (from original train/)...
 SOURCE split: train=2776, val=595, test=595

Cleaning up original RoboFlow folders...


In [20]:
dataset_dir = Path.cwd()

#edit source and target sets yaml file to correct path
for directory in ("source", "target"):
    
    yaml_path = dataset_dir / directory / "data.yaml"
    
    cfg = {
        "path": str(dataset_dir / directory),
        "train": "train/images",
        "val":   "val/images",
        "nc": 7,
        "names": [
            "Birdnest",
            "Broken_Insulator",
            "Defective_Damper",
            "Flashover_Insulator",
            "Normal_Damper",
            "Normal_Insulators",
            "Self-Exploded_Insulator",
        ],
    }

    # add a test split only for the target set
    # if directory == "target":
    cfg["test"] = "test/images"

    yaml_path.parent.mkdir(parents=True, exist_ok=True)
    yaml_path.write_text(yaml.safe_dump(cfg, sort_keys=False))
    print("Created", yaml_path.relative_to(dataset_dir))

Created source/data.yaml
Created target/data.yaml


In [21]:
for directories in ("source", "target"):
    for split in ("train", "val", "test"):
        p = dataset_dir / directories / split
        if os.path.exists(p):
            image_pairs = get_image_label_pairs(p) 
            print(f"Found {len(image_pairs)} images in {p}.")


Found 1057 images in /mnt/sdb/home/si25_osts0525/UAV_Inspection_System/dataset/source/train.
Found 132 images in /mnt/sdb/home/si25_osts0525/UAV_Inspection_System/dataset/source/val.
Found 133 images in /mnt/sdb/home/si25_osts0525/UAV_Inspection_System/dataset/source/test.
Found 814 images in /mnt/sdb/home/si25_osts0525/UAV_Inspection_System/dataset/target/train.
Found 174 images in /mnt/sdb/home/si25_osts0525/UAV_Inspection_System/dataset/target/val.
Found 175 images in /mnt/sdb/home/si25_osts0525/UAV_Inspection_System/dataset/target/test.


In [2]:
## -------------------When Only Focus on Target Dataset-------------------

%cd dataset
os.mkdir("target")

/mnt/sdb/home/si25_osts0525/UAV_Inspection_System/dataset


In [3]:
%cd target

/mnt/sdb/home/si25_osts0525/UAV_Inspection_System/dataset/target


In [ ]:
#after importing train set, delete data.yaml and the  read.me, rename
# !curl -L "https://app.roboflow.com/ds/x11KtfmSz2?key=ZimtpHyFTc" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip
!curl -L "https://app.roboflow.com/ds/LpLKuppXPW?key=ovp2NQzcqz" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip

In [ ]:
#change the train directory name then import validation set then change the directroy name to val, and remove the data.yaml and readme files
#!curl -L "https://app.roboflow.com/ds/k8sUaw1BGw?key=zLRPwyzJF6" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip
!curl -L "https://app.roboflow.com/ds/Eln2rYJVxJ?key=PmrX8RRkvV" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip

In [ ]:
#import test set then change the directroy name to test, and remove the data.yaml and readme files
!curl -L "https://app.roboflow.com/ds/30CkBbRhfb?key=9AWf2EbnPq" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip

In [11]:
dataset_dir = Path.cwd()

#edit target sets yaml file to correct path
yaml_path = dataset_dir / "data.yaml"
cfg = {
        "path": str(dataset_dir),
        "train": "train/images",
        "val":   "val/images",
        "nc": 7,
        "names": [
            "Birdnest",
            "Broken_Insulator",
            "Defective_Damper",
            "Flashover_Insulator",
            "Normal_Damper",
            "Normal_Insulators",
            "Self-Exploded_Insulator",
        ],
}

# add a test split only for the target set
# if directory == "target":
cfg["test"] = "test/images"

yaml_path.parent.mkdir(parents=True, exist_ok=True)
yaml_path.write_text(yaml.safe_dump(cfg, sort_keys=False))
print("Created", yaml_path.relative_to(dataset_dir))

Created data.yaml
